In [ ]:
#1
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split 
import statsmodels.api as sm
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [ ]:
#2
os.chdir('your file location')

# loading Training data

In [ ]:
#3
x = pd.read_csv("train.csv")

In [ ]:
#4
df = x
#x=df

In [ ]:
#5
x.head()

## preaparing data

In [ ]:
#6
x = x.set_index('ID_code')
target = x['target']
x = pd.concat([x.iloc[:,1:201],target], axis = 1)
x['target'] = x['target'].astype('category')

# Data Pre-Processing

## Missing Value Analysis

In [ ]:
#7
mval = pd.DataFrame(x.isnull().sum())
mval = mval.rename(columns ={0 :'x'})
mval = mval.reset_index()
mval = mval.sort_values('x', ascending= False).reset_index(drop = True)
mval.head()

## outlier Analysis

In [ ]:
#8
#Plot boxplot to visualize Outliers
%matplotlib inline  
plt.boxplot(x['var_98'])
#plt.boxplot(x['var_3'])

In [ ]:
#9
for i in range(200):
    print (i)
    q75,q25 = np.percentile(x.iloc[:,i], [75,25])
    iqr = q75 - q25
    
    min = q25 - 1.5*iqr
    max = q75 + 1.5*iqr
    
    x = x.drop(x[x.iloc[:,i] < min].index)
    x = x.drop(x[x.iloc[:,i] > max].index)

In [ ]:
x.shape

## Feature Selection

In [ ]:
#10
f, ax = plt.subplots(figsize =(10,8))
corr = x[0:200].corr()
sb.heatmap(corr, mask=np.zeros_like(corr,dtype=np.bool8 ), cmap=sb.diverging_palette(10, 20))

In [ ]:
x['target'].describe()

# Feature Scaling

In [ ]:
#11
for i in range(200):
    print(i)
    x.iloc[:,i] =((x.iloc[:,i] - x.iloc[:,i].min())/(x.iloc[:,i].max()-x.iloc[:,i].min())) 

In [ ]:
x.iloc[13,200]

In [ ]:
x.tail(4)

## Feature Sampling

In [ ]:
#12
train, test = train_test_split(x, test_size = 0.3)

In [ ]:
train.shape


# Model Develoment

 ## Logistic Regression

In [ ]:
#13
tcol = train.columns[1:200]
logit = sm.Logit(train['target'], train[tcol]).fit()

In [ ]:
logit.summary()

In [ ]:
#14
test['Actual_prob'] = logit.predict(test[tcol])
test.loc[test.Actual_prob > 0.5, 'actual value'] = 1
test.loc[test.Actual_prob < 0.5, 'actual value'] = 0
#test['ActualVal'] = 1
#test.loc[test.Actual_prob < 0.5, 'ActualVal'] = 0

In [ ]:
#15
fpr, tpr, thresholds = metrics.roc_curve(test['target'], test['actual value'])
auc = roc_auc_score(test['target'],test['Actual_prob'])
# This is the ROC curve
plt.plot(fpr, tpr,label="ROC, AUC = "+str(auc))
plt.legend(loc=4)

In [ ]:
#16
CM = pd.crosstab(test['target'], test['actual value'])
CM

In [ ]:
#17
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]


In [ ]:
#18
accuracy = ((TP+TN)*100)/(TP+TN+FP+FN)
FNR = (FN*100/(FN+TN))
FPR = (FP*100/(FP+TP))
Precision = (TP*100/(FP+TP))
Recall = (TP*100/(FN+TP))

#accuracy = 91.74%
#FNR = 7.38%
#FPR = 31.386%
#Precision = 68.61%
#Recall = 26.40%

## Desicion Tree

In [ ]:
#19
x_indi = train.iloc[:,0:200]
x_dep = train.iloc[:,200]
y_indi = test.iloc[:,0:200]
y_dep = test.iloc[:,200]
C50_model = tree.DecisionTreeClassifier(criterion='entropy').fit(x_indi, x_dep)
C50_Predictions = C50_model.predict(y_indi)

In [ ]:
#20
fpr, tpr, thresholds = metrics.roc_curve(y_dep, C50_Predictions)
auc = roc_auc_score(y_dep, C50_Predictions)
# This is the ROC curve
plt.plot(fpr, tpr,label="ROC, AUC = "+str(auc))
plt.legend(loc=4)

In [ ]:
#21
CM = pd.crosstab(y_dep, C50_Predictions)
CM

In [ ]:
#22
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

In [ ]:
#23
accuracy = ((TP+TN)*100)/(TP+TN+FP+FN)
FNR = (FN*100/(FN+TN))
FPR = (FP*100/(FP+TP))
Precision = (TP*100/(FP+TP))
Recall = (TP*100/(FN+TP))

#accuracy = 83.82%
#FNR = 8.80%
#FPR = 81.63%
#Precision = 18.37%
#Recall = 19.48%

## KNN 

In [ ]:
#24
x_indi = train.iloc[:,0:200]
x_dep = train.iloc[:,200]
y_indi = test.iloc[:,0:200]
y_dep = test.iloc[:,200]
KNN_model = KNeighborsClassifier(n_neighbors = 3).fit(x_indi, x_dep)
KNN_Predictions = KNN_model.predict(y_indi)

In [ ]:
#25
fpr, tpr, thresholds = metrics.roc_curve(y_dep, KNN_Predictions)
auc = roc_auc_score(y_dep, KNN_Predictions)
# This is the ROC curve
plt.plot(fpr, tpr,label="ROC, AUC = "+str(auc))
plt.legend(loc=4)

In [ ]:
#26
CM = pd.crosstab(y_dep, KNN_Predictions)
CM

In [ ]:
KNN_Predictions.dtype

In [ ]:
#27
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

In [ ]:
#28
accuracy = ((TP+TN)*100)/(TP+TN+FP+FN)
FNR = (FN*100/(FN+TN))
FPR = (FP*100/(FP+TP))
Precision = (TP*100/(FP+TP))
Recall = (TP*100/(FN+TP))

#accuracy = 83.82%
#FNR = 8.80%
#FPR = 81.63%
#Precision = 18.37%
#Recall = 19.01%


# Test Data



In [ ]:
#29
y = pd.read_csv('test.csv')
y = y.set_index('ID_code')

In [ ]:
#30
#bu = y

y = bu

In [ ]:
y.head()

## scaling test data

In [ ]:
#31
for i in range(200):
    print(i)
    y.iloc[:,i] =((y.iloc[:,i] - y.iloc[:,i].min())/(y.iloc[:,i].max()-y.iloc[:,i].min())) 

## prediction

In [ ]:
#32
y['Actual_prob'] = logit.predict(y[tcol])
y.loc[y.Actual_prob < 0.5, 'Test_target'] = 0
y.loc[y.Actual_prob > 0.5, 'Test_target'] = 1
y['Test_target'] = y['Test_target'].astype('int')
y['Test_target'] = y['Test_target'].astype('category')
y.head()

In [ ]:
y['Test_target'].value_counts()

In [ ]:
#33
Test_target = y['Test_target']
y = y.drop(labels = 'Test_target',axis =1)
y = y.drop(labels = 'Actual_prob',axis =1)
y = pd.concat([Test_target, y], axis = 1)

In [ ]:
#34
y.to_csv("Pr.csv", index = True)